### Part 1

#### Comparison of SNAFU to base-ten

Base ten:

1,580 -> 10^3 * 1 + 10^2 * 5 + 10^1 * 8 + 10^0 * 0

In [1]:
10**3 + 5 * 10**2 + 8 * 10 ** 1 + 0 * 10 ** 0

1580

SNAFU swaps out the 10 with a 5, so should be straight forward using power logic.

Also uses (2,1,0,-,=)


### Approach: 
- I first built a `snafu -> decimal` function, this was helpful to confirm I understood the logic and helped me to do some checks
- Next, I could actually avoid converting from `decimal` to `SNAFU`. Instead I can just sum up directly with a little bit of modulus math. 

In [2]:
# map a string SNAFU -> base-10
snafu_map = {
    '2': 2, '1': 1, '0': 0, '-': -1, '=': -2
}

# map a base-10 to SNAFU
dec_to_snafu = {v:k for k,v in snafu_map.items()}

In [3]:
def snafu_to_dec(string, snafu_map):
    powers = list(range(len(string)))
    total = 0
    for v, p in zip(string, reversed(powers)):
        total += int(snafu_map[v]) * (5 ** p)
    
    return total

In [4]:
tests = ["1=11-2", "1=0", "1121-1110-1=0"]
solutions = [2022, 15, 314159265]

# convert to base 10 from SNAFU
for idx in range(len(tests)):
    assert snafu_to_dec(tests[idx], snafu_map) == solutions[idx]

# and we can sum & check
assert (snafu_to_dec(tests[0], snafu_map) + snafu_to_dec(tests[1], snafu_map)) == solutions[0] + solutions[1]

In [5]:
# this was determined by using the "snafu_to_dec" function above
mod_map = {5: (1, '0'), 4: (1, '-'), 3: (1, '='), 
           -3: (-1, '2'), -4: (-1, '1'), -5: (-1, '0')}

# Can keep everything in SNAFU, solve right to left
def snafu_addition(string1, string2, snafu_map):
    
    s1 = len(string1)
    s2 = len(string2)
    
    # pad the shorter one w/ 0s as strings
    if s1 > s2:
        string2 = string2.rjust(s1, '0')
    else:
        string1 = string1.rjust(s2, '0')
        
    #print(string1)
    #print(string2)
    
    r = 0
    string = ''
    for i, (t, b) in enumerate(zip(reversed(string1), reversed(string2))):
        #print(i,t,b) start by unpacking the index, and each char
        
        # Calculate a decimal total
        total = snafu_map[t] + snafu_map[b] + r
        #print(total)
        
        # our system is limited to -2 -> 2 boundary, anything else must be handled 
        # using mod math
        if -2 <= total <= 2:
            #print("can represent")
            string += dec_to_snafu[total]
            r = 0
            
        # hardcoded mod math above, going char by char
        else:
            #print("need some mod math")
            r, char = mod_map[total]
            string += char
        
    if r > 0:
        string+= str(r)
        
    return string[::-1]

In [6]:
# quick test
snafu_out = snafu_addition(tests[0], tests[1], snafu_map)
assert snafu_to_dec(snafu_out, snafu_map) == solutions[0] + solutions[1]

In [7]:
# sample -> sum line by line
with open("data/day25_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
snafu_out = snafu_addition(lines[0], lines[1], snafu_map)
for i in range(2, len(lines)):
    snafu_out = snafu_addition(snafu_out, lines[i], snafu_map)

print(snafu_out)

2=-1=0


In [8]:
# sample -> sum line by line
with open("data/day25.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
snafu_out = snafu_addition(lines[0], lines[1], snafu_map)
#print(snafu_to_dec(lines[0], snafu_map) + (snafu_to_dec(lines[1], snafu_map)))
cumu_list = []
for i in range(2, len(lines)):
    snafu_out = snafu_addition(snafu_out, lines[i], snafu_map)
    cumu_list.append(snafu_out)
    #print(i, snafu_to_dec(snafu_out, snafu_map))

print(snafu_out)

2=222-2---22=1=--1-2


In [9]:
# And we can just do a quick confirmation
sum_v = 0
snaf_list = []
for i in lines:
    #print(i)
    v = snafu_to_dec(i, snafu_map)
    sum_v += v
    snaf_list.append(sum_v)

assert(sum_v == snafu_to_dec(snafu_out, snafu_map))